In [28]:
from urllib.parse import urljoin

import bs4
import requests
import pandas as pd

eia_906_url = "https://www.eia.gov/electricity/data/eia923/eia906u.php"
response = requests.get(eia_906_url)
soup = bs4.BeautifulSoup(response.text)

a_tags = soup.find_all("a", href=True)
eia_906_links = []
for a in a_tags:
    if ".xls" in a["href"].lower():
        eia_906_links.append(a)

eia_906_dataframes = []
for a in eia_906_links:
    full_url = urljoin(eia_906_url, a["href"])
    eia_906_dataframes.append(pd.read_excel(full_url))

In [31]:
import functools

intersection = functools.reduce(lambda acc, ix: acc.intersection(ix), [df.columns for df in eia_906_dataframes])
print(intersection)
print(eia_906_dataframes[0].columns)

Index(['CENSUS', 'FIPST', 'OWNER', 'PMOVER', 'FUELTYP', 'COCODE', 'PLTCODE',
       'UTILNAME', 'PLTNAME', 'CAPACITY', 'FUELNM', 'EFFDATE', 'STATUS',
       'MULTIST', 'YEAR', 'PCODE', 'NERC', 'UTILCODE', 'FUELDESC', 'PMDESC'],
      dtype='object')
Index(['CENSUS', 'FIPST', 'OWNER', 'PMOVER', 'FUELTYP', 'COCODE', 'PLTCODE',
       'UTILNAME', 'PLTNAME', 'CAPACITY', 'FUELNM', 'UCODE', 'FILLER',
       'FREQUENCYF', 'EFFDATE', 'STATUS', 'MULTIST', 'YEAR', 'GEN01', 'CON01',
       'STK01', 'GEN02', 'CON02', 'STK02', 'GEN03', 'CON03', 'STK03', 'GEN04',
       'CON04', 'STK04', 'GEN05', 'CON05', 'STK05', 'GEN06', 'CON06', 'STK06',
       'GEN07', 'CON07', 'STK07', 'GEN08', 'CON08', 'STK08', 'GEN09', 'CON09',
       'STK09', 'GEN10', 'CON10', 'STK10', 'GEN11', 'CON11', 'STK11', 'GEN12',
       'CON12', 'STK12', 'PCODE', 'NERC', 'UTILCODE', 'FUELDESC', 'PMDESC'],
      dtype='object')


In [40]:
pd.concat(eia_906_dataframes).info()

<class 'pandas.core.frame.DataFrame'>
Index: 198632 entries, 0 to 6099
Data columns (total 64 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   CENSUS      197012 non-null  float64
 1   FIPST       198632 non-null  int64  
 2   OWNER       185854 non-null  float64
 3   PMOVER      197012 non-null  float64
 4   FUELTYP     143911 non-null  object 
 5   COCODE      198632 non-null  int64  
 6   PLTCODE     198619 non-null  float64
 7   UTILNAME    198631 non-null  object 
 8   PLTNAME     196867 non-null  object 
 9   CAPACITY    197012 non-null  float64
 10  FUELNM      143047 non-null  object 
 11  UCODE       171475 non-null  float64
 12  FILLER      176594 non-null  object 
 13  FREQUENCYF  171475 non-null  object 
 14  EFFDATE     197012 non-null  float64
 15  STATUS      14706 non-null   object 
 16  MULTIST     66731 non-null   object 
 17  YEAR        198632 non-null  int64  
 18  GEN01       173682 non-null  float64
 19  CON01    

In [114]:
import math
import requests

base_url = "https://api.eia.gov/v2/electricity"
api_key = "3zjKYxV86AqtJWSRoAECir1wQFscVu6lxXnRVKG8"

# remember from last time that the shape is a JSON object with everything in "response"
net_generation = requests.get(
  f"{base_url}/facility-fuel/data",
  params={
    "data[]": "generation",
    "facets[state][]": "CO",
    "api_key": api_key
  }
).json()["response"]

total_rows = int(net_generation["total"])
page_size = 5000
num_pages = math.ceil(total_rows / page_size)

all_records = []

for page_num in range(num_pages):
    offset = page_num * page_size
    print(offset)
    page = requests.get(
      f"{base_url}/facility-fuel/data",
      params={
        "data[]": "generation",
        "facets[state][]": "CO",
        "sort[][column]": "period",
        "sort[][direction]": "desc",
        "offset": offset,
        "api_key": api_key
      }
    ).json()["response"]["data"]
    all_records.extend(page)

0


KeyError: 'response'

In [116]:
first_page = requests.get(
  f"{base_url}/facility-fuel/data",
  params={
    "data[]": "generation",
    "facets[state][]": "CO",
    "sort[0][column]": "period",
    "sort[0][direction]": "desc",
    "api_key": api_key
  }
).json()["response"]

next_page = requests.get(
  f"{base_url}/facility-fuel/data",
  params={
    "data[]": "generation",
    "facets[state][]": "CO",
    "sort[0][column]": "period",
    "sort[0][direction]": "desc",
    "offset": 5000,
    "api_key": api_key
  }
).json()["response"]


pd.DataFrame(first_page["data"] + next_page["data"]).drop_duplicates()

,period,plantCode,plantName,fuel2002,fuelTypeDescription,state,stateDescription,primeMover,generation,generation-units
0,2024-12,61991,"Pioneer Solar (CO), LLC",ALL,Total,CO,Colorado,ALL,9063,megawatthours
1,2024-12,6619,Burlington (CO),DFO,Distillate Fuel Oil,CO,Colorado,GT,693,megawatthours
2,2024-12,6619,Burlington (CO),DFO,Distillate Fuel Oil,CO,Colorado,ALL,693,megawatthours
3,2024-12,6619,Burlington (CO),ALL,Total,CO,Colorado,ALL,693,megawatthours
4,2024-12,62840,"Mountain Breeze Wind, LLC",WND,Wind,CO,Colorado,WT,59585,megawatthours
...,...,...,...,...,...,...,...,...,...,...
9995,2023-05,54142,Hillcrest Pump Station,WAT,Hydroelectric Conventional,CO,Colorado,ALL,281,megawatthours
9996,2023-05,54142,Hillcrest Pump Station,ALL,Total,CO,Colorado,ALL,281,megawatthours
9997,2023-05,55200,Arapahoe Combustion Turbine Project,NG,Natural Gas,CO,Colorado,CT,8200,megawatthours
9998,2023-05,55200,Arapahoe Combustion Turbine Project,NG,Natural Gas,CO,Colorado,CA,1784,megawatthours


In [113]:
pd.DataFrame(all_records).drop_duplicates()

,period,plantCode,plantName,fuel2002,fuelTypeDescription,state,stateDescription,primeMover,generation,generation-units
0,2023-08,66095,"Cortez Solar 1, LLC",ALL,Total,CO,Colorado,ALL,571,megawatthours
1,2023-08,66095,"Cortez Solar 1, LLC",SUN,Solar,CO,Colorado,ALL,571,megawatthours
2,2023-08,66095,"Cortez Solar 1, LLC",SUN,Solar,CO,Colorado,PV,571,megawatthours
3,2009-02,6206,Tacoma,ALL,Total,CO,Colorado,ALL,1041.31,megawatthours
4,2009-02,6206,Tacoma,WAT,Hydroelectric Conventional,CO,Colorado,ALL,1041.31,megawatthours
...,...,...,...,...,...,...,...,...,...,...
154949,2017-10,59656,Comanche Solar,SUN,Solar,CO,Colorado,ALL,25047.29,megawatthours
154950,2017-10,59656,Comanche Solar,SUN,Solar,CO,Colorado,PV,25047.29,megawatthours
154951,2024-01,61811,Titan Solar,ALL,Total,CO,Colorado,ALL,5868,megawatthours
154952,2024-01,61811,Titan Solar,SUN,Solar,CO,Colorado,ALL,5868,megawatthours


In [ ]:
while_pages = []
page_num = 0
last_page_size = None
print(f"total is allegedly {total_rows}")
while (last_page_size != 0) and (page_num < 50):
    offset = page_num * page_size
    print(f"getting {offset}:{offset + page_size}")
    page = requests.get(
      f"{base_url}/facility-fuel/data",
      params={
        "data[]": "generation",
        "facets[state][]": "CO",
        "sort[0][column]": "period",
        "sort[0][direction]": "desc",
        "offset": offset,
        "api_key": api_key
      }
    ).json()["response"]
    last_page_size = len(page["data"])
    print(f"got {last_page_size} rows")
    while_pages.extend(page["data"])
    page_num += 1


getting 0:5000
total is allegedly 155130
getting 5000:10000
total is allegedly 155130
getting 10000:15000
total is allegedly 155130
getting 15000:20000
total is allegedly 155130
getting 20000:25000
total is allegedly 155130
getting 25000:30000
total is allegedly 155130
getting 30000:35000
total is allegedly 155130
getting 35000:40000
total is allegedly 155130
getting 40000:45000
total is allegedly 155130
getting 45000:50000
total is allegedly 155130
getting 50000:55000
total is allegedly 155130
getting 55000:60000
total is allegedly 155130
getting 60000:65000
total is allegedly 155130
getting 65000:70000
total is allegedly 155130
getting 70000:75000
total is allegedly 155130
getting 75000:80000
total is allegedly 155130
getting 80000:85000
total is allegedly 155130
getting 85000:90000
total is allegedly 155130
getting 90000:95000
total is allegedly 155130
getting 95000:100000
total is allegedly 155130
getting 100000:105000
total is allegedly 155130
getting 105000:110000
total is allege

KeyboardInterrupt: 

In [73]:
pd.DataFrame(net_generation["data"]).period.value_counts()

period
2006-03    186
2008-06    186
2011-06    178
2019-02    169
2008-07    148
          ... 
2019-12      3
2010-08      3
2018-06      3
2022-12      3
2014-08      2
Name: count, Length: 140, dtype: int64

In [80]:
next_page = requests.get(
  f"{base_url}/facility-fuel/data",
  params={
    "data[]": "generation",
    "facets[state][]": "CO",
    "sort[0][column]": "period",
    "sort[0][direction]": "desc",
    "offset": 5000,
    "api_key": api_key
  }
).json()["response"]

In [82]:
combined = pd.DataFrame(net_generation["data"] + next_page["data"])

In [83]:
len(combined)

10000

In [97]:
combined[combined.duplicated(keep=False) & (combined.plantCode == "55453")]

,period,plantCode,plantName,fuel2002,fuelTypeDescription,state,stateDescription,primeMover,generation,generation-units
4384,2023-11,55453,Fountain Valley Power Facility,NG,Natural Gas,CO,Colorado,GT,2818,megawatthours
4385,2023-11,55453,Fountain Valley Power Facility,NG,Natural Gas,CO,Colorado,ALL,2818,megawatthours
4386,2023-11,55453,Fountain Valley Power Facility,ALL,Total,CO,Colorado,ALL,2818,megawatthours
5084,2023-11,55453,Fountain Valley Power Facility,NG,Natural Gas,CO,Colorado,GT,2818,megawatthours
5085,2023-11,55453,Fountain Valley Power Facility,NG,Natural Gas,CO,Colorado,ALL,2818,megawatthours
5086,2023-11,55453,Fountain Valley Power Facility,ALL,Total,CO,Colorado,ALL,2818,megawatthours
